In [1]:
import sys
sys.path.insert(0, "/Users/office/Documents/portfolio")
import sembleu
import os, json, math, random, time

from collections import Counter
from pathlib import Path

from sembleu import src
from sembleu.src import bleu_score
from sembleu.src.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction, NgramInst

from sembleu.src import amr_graph
from sembleu.src.amr_graph import AMRGraph

import matplotlib.pyplot as plt
import numpy as np

import fractions
try:
    fractions.Fraction(0, 1000, _normalize=False)
    from fractions import Fraction
except TypeError:
    from nltk.compat import Fraction

from collections import namedtuple
NgramInst = namedtuple('NgramInst', 'ngram length')

#from transformers import LlamaTokenizer

%matplotlib inline

In [2]:
dir_ = Path('/Users/office/Documents/AMRs-2022/massive-qa-amrs/data/')


In [61]:
def read_amr(path):
    id_dict = {}
    amrs, id_list, ids, sents = [],[],[],[]
    amr_str = ''
    for line in open(path,'r'):
        
        if '::en_utt' in line or '::annot_utt' in line:
            continue
        
#         if line.startswith('#'):
#             if line.startswith('# ::id'):
#                 id = line.strip().split()[2]
#                 ids.append(id)
#                 id_dict[id] = len(ids)-1
#                 id_list.append(id)
#             if line.startswith('# ::snt'):
#                 snt = line[2:].strip().replace('::snt ', '')
#                 sents.append(snt)
#             continue
            

            
        elif '::id' in line or '::snt' in line:
            if line.startswith('::id'):
                id = line.strip().split()[1]
                ids.append(id)
                id_dict[id] = len(ids)-1
                id_list.append(id)
            if line.startswith('::snt'):
                snt = line.strip().replace('::snt ', '')
                sents.append(snt)
            continue

        line = line.strip()
        if line == '':
            if amr_str != '':
                amrs.append(amr_str.strip())
                amr_str = ''
        else:
            amr_str = amr_str + line + ' '

    if amr_str != '':
        amrs.append(amr_str.strip())
        amr_str = ''
        
    return (amrs, ids, sents), id_list

def get_amr_ngrams(path, max_ngrams=3, stat_save_path=None, raw=False, id_list_tmp=list()):
    
    data, raw_data, sentences = list(), list(), list()

    if stat_save_path:
        f = open(stat_save_path, 'w')

    (lines, amrids, sents), id_list = read_amr(path) 
    
    if len(amrids)==0:
        amrids = id_list_tmp
        
    for line, amrid, snt in zip(lines, amrids, sents):
        
        if line:
            raw_data.append(line)
            sentences.append(snt)
            try:
                amr = AMRGraph(line.strip())
            except AssertionError:
                assert False
            amr.revert_of_edges()
            ngrams = amr.extract_ngrams(max_ngrams, multi_roots=True) # dict(list(tuple))
            
            length = 0
            for k, v in ngrams.items():
                length+=len(v)
            #data.append(NgramInst(ngram=ngrams, length=len(amr.edges)))
            data.append(NgramInst(ngram=ngrams, length=length))
            if stat_save_path:
                print(len(amr), len(ngrams[1]), len(ngrams[2]), len(ngrams[3]), file=f)

    if stat_save_path:
        f.close()
    if raw:
        return data, raw_data, sentences, id_list
    else:
        return data, id_list

In [62]:
def get_amr_data(directory, max_ngrams=3):
    
    train_instances, valid_instances = list(), list()
    
    fnames = ['amrs-massive-val', 'amrs-massive-train']
    
    for fname in fnames:
        
        for pth in directory.glob('**/*'):

            if fname in pth.name:
                print(pth.name)
                
                ngramInstances, hypotheses, raw_data, sentences, ids = list(), list(), list(), list(), list()

                h, r, s, i = get_amr_ngrams(pth, max_ngrams, raw=True)

    #             amr_ngrams_filtered_temp = [item for item in h.ngram[2] if ':ARG' in item[1]]
    
                if max_ngrams == 1:
                    amr_ngrams_filtered = [h[i].ngram[1] for i in range(len(h))]
                elif max_ngrams == 2:
                    amr_ngrams_filtered = [h[i].ngram[1] + h[i].ngram[2] for i in range(len(h))]
                else:
                    amr_ngrams_filtered = [h[i].ngram[1] + h[i].ngram[2] + h[i].ngram[3] for i in range(len(h))]
                    
                ngramInstances.extend(h)

                # consider filtering out a few degenerate utts of single tokens
                hypotheses.extend(amr_ngrams_filtered)
                raw_data.extend(r)
                sentences.extend(s)
                ids.extend(i)
            
                for ngramInst, hypo, raw_amr, sent, identifier in zip(ngramInstances, hypotheses, raw_data, sentences, ids):

                    if 'en-US' in identifier and len(sent.split())>1:
                        
                        thisInstance = {'question': sent, 'amr_ngrams': hypo, 'ngramInstance': ngramInst, 'raw_amr': raw_amr, 'id': identifier}

                        if 'val' in fname:
                            valid_instances.append(thisInstance)
                        else:
                            train_instances.append(thisInstance)
                    
    return train_instances, valid_instances
            
        

In [63]:
def get_rand_idx(num_samples, train_instances):

    rand_idxs = list()
    for _ in range(num_samples):
        rand_idxs.append(random.randint(0, len(train_instances)))

    return list(set(rand_idxs))

In [64]:
def get_prompts_list(max_ngrams=3, num_samples=8):
    
    these_amr_prompts = []
    
    #train_sample = random.sample(train_instances, 12)
    
    targets = ['amr_ngrams', 'raw_amr']
            
    train_instances, valid_instances = get_amr_data(dir_, max_ngrams=max_ngrams)
    
    rand_idxs = get_rand_idx(num_samples, train_instances)
    
    train_sample = list()
    
    for thisIdx in rand_idxs:
    
        train_sample.append(train_instances[thisIdx])
    
    for target in targets:
     
        for idx, thisInstance in enumerate(valid_instances):

            if target=='amr_ngrams':
                tgt_string = 'List of semantic frames'
            else:
                tgt_string = 'Abstract Meaning Representation (AMR)'

            amr_dialog = [
                        {"role": "system", "content": f"Decompose the utterance into a {tgt_string} like in the examples."}
                    ]
            
            for sample in train_sample:
                amr_dialog.append({"role": "user", "content": f"Utterance: {sample['question']}\n{tgt_string}: "})
                amr_dialog.append({"role": "assistant", "content": f"{sample[target]}"})

            finalUserTurn = {"role": "user", "content": f"Utterance: {thisInstance['question']}\n{tgt_string}: "}

            amr_dialog.append(finalUserTurn)

            thisInstance = {"dialog": amr_dialog, 
                            "question": thisInstance['question'], 
                            "target": target,
                            "amr_ngrams": thisInstance['amr_ngrams'],
                            "ngramInstance": thisInstance['ngramInstance'],
                            "max_ngrams": max_ngrams,
                            "num_samples_in_prompt": num_samples,
                           "raw_amr": thisInstance['raw_amr'],
                           "id": thisInstance['id']}
            these_amr_prompts.append(thisInstance)
            
    return these_amr_prompts

In [65]:
massive_prompts_1_8 = get_prompts_list(max_ngrams=1, num_samples=8)

len(massive_prompts_1_8)

amrs-massive-val.txt
amrs-massive-train.txt


84

In [66]:
massive_prompts_1_8[0]

{'dialog': [{'role': 'system',
   'content': 'Decompose the utterance into a List of semantic frames like in the examples.'},
  {'role': 'user',
   'content': "Utterance: what's two plus two\nList of semantic frames: "},
  {'role': 'assistant',
   'content': "[('equal-01',), ('sum-of',), ('amr-unknown',), ('2',), ('2',)]"},
  {'role': 'user',
   'content': 'Utterance: did angelina jolie leave brad pitt\nList of semantic frames: '},
  {'role': 'assistant',
   'content': "[('leave-15',), ('amr-unknown',), ('person',), ('person',), ('name',), ('name',), ('angelina',), ('jolie',), ('brad',), ('pitt',)]"},
  {'role': 'user',
   'content': "Utterance: what's three plus three hundred and fifty five\nList of semantic frames: "},
  {'role': 'assistant',
   'content': "[('equal-01',), ('sum-of',), ('amr-unknown',), ('355',), ('3',)]"},
  {'role': 'user',
   'content': 'Utterance: how far away is the sun\nList of semantic frames: '},
  {'role': 'assistant',
   'content': "[('far-01',), ('star',),

In [76]:
len(massive_prompts_1_8[0]['ngramInstance'][0][1])

#massive_prompts_1_8[0]['ngramInstance'][0][1]

5

In [67]:
massive_prompts_1_8[-1]

{'dialog': [{'role': 'system',
   'content': 'Decompose the utterance into a Abstract Meaning Representation (AMR) like in the examples.'},
  {'role': 'user',
   'content': "Utterance: what's two plus two\nAbstract Meaning Representation (AMR): "},
  {'role': 'assistant',
   'content': '(e / equal-01 :ARG1 (s / sum-of :op1 2 :op2 2) :ARG2 (a / amr-unknown))'},
  {'role': 'user',
   'content': 'Utterance: did angelina jolie leave brad pitt\nAbstract Meaning Representation (AMR): '},
  {'role': 'assistant',
   'content': '(l / leave-15 :ARG0 (p / person :name (n / name :op1 "angelina" :op2 "jolie")) :ARG1 (p2 / person :name (n2 / name :op1 "brad" :op2 "pitt")) :polarity (a / amr-unknown))'},
  {'role': 'user',
   'content': "Utterance: what's three plus three hundred and fifty five\nAbstract Meaning Representation (AMR): "},
  {'role': 'assistant',
   'content': '(e / equal-01 :ARG1 (s / sum-of :op1 3 :op2 355) :ARG2 (a / amr-unknown))'},
  {'role': 'user',
   'content': 'Utterance: how 

In [68]:
write_path = Path('../data/llama-massive-prompts_2023-08-07.json')

with open(write_path, 'w') as fout:
    #data = json.dumps(massive_prompts_1_8)
    json.dump(massive_prompts_1_8, fout, indent=4)

In [23]:
thisHyp = {1: [('share-border-91',), ('border',), ('amr-unknown',), ('country',), ('name',), ('spain',), ('morocco',)]}
thisRef = {1: [('border-01',), ('amr-unknown',), ('country',), ('country',), ('name',), ('name',), ('spain',), ('morrocco',)]} 

cnt_match_ngrams = 0
for ngram in thisHyp[1]:
    if ngram in thisRef[1]:
        cnt_match_ngrams+=1
#print(f"Sembleu: {cnt_match_ngrams/len(thisRef)}")

weights = (1.0,)
smoofunc = getattr(SmoothingFunction(), 'method3')

thisHyp = NgramInst(ngram=thisHyp, length=1)
thisRef = NgramInst(ngram=thisRef, length=1)


In [24]:
thisHyp.ngram[1]

[('share-border-91',),
 ('border',),
 ('amr-unknown',),
 ('country',),
 ('name',),
 ('spain',),
 ('morocco',)]

In [26]:
def brevity_penalty(closest_ref_len, hyp_len):
    if hyp_len > closest_ref_len:
        return 1.0
    # If hypothesis is empty, brevity penalty = 0 should result in BLEU = 0.0
    elif hyp_len == 0:
        return 0.0
    else:
        return math.exp(1 - closest_ref_len / hyp_len)

def closest_ref_length_amr(references, hyp_len):
    """
    This function finds the reference that is the closest length to the
    hypothesis. The closest reference length is referred to as *r* variable
    from the brevity penalty formula in Papineni et. al. (2002)

    :param references: A list of reference translations.
    :type references: list(list(str))
    :param hypothesis: The length of the hypothesis.
    :type hypothesis: int
    :return: The length of the reference that's closest to the hypothesis.
    :rtype: int
    """
    ref_lens = (reference.length for reference in references)
    closest_ref_len = min(ref_lens, key=lambda ref_len:
                          (abs(ref_len - hyp_len), ref_len))
    return closest_ref_len

def modified_precision_amr(references, hypothesis, n):
    # Extracts all ngrams in hypothesis
    # Set an empty Counter if hypothesis is empty.
    counts = Counter(hypothesis.ngram[n]) if n in hypothesis.ngram else Counter()
    #print 'counts', counts
    # Extract a union of references' counts.
    ## max_counts = reduce(or_, [Counter(ngrams(ref, n)) for ref in references])
    max_counts = {}
    for reference in references:
        reference_counts = Counter(reference.ngram[n]) if n in reference.ngram else Counter()
        for ngram in counts:
            max_counts[ngram] = max(max_counts.get(ngram, 0),
                                    reference_counts[ngram])
    #print('max_counts', max_counts)

    # Assigns the intersection between hypothesis and references' counts.
    clipped_counts = {ngram: min(count, max_counts[ngram])
                      for ngram, count in list(counts.items())}
    #print('clipped_counts', clipped_counts)

    numerator = sum(clipped_counts.values())
    denominator = sum(counts.values())
    ## Ensures that denominator is minimum 1 to avoid ZeroDivisionError.
    ## Usually this happens when the ngram order is > len(reference).
    #denominator = max(1, sum(counts.values()))

    if denominator == 0:
        return None

    return Fraction(numerator, denominator, _normalize=False)


def corpus_bleu(list_of_references, hypotheses, weights=(0.34, 0.33, 0.33),
                smoothing_function=None, auto_reweigh=False,
                emulate_multibleu=False):
    # Before proceeding to compute BLEU, perform sanity checks.

    p_numerators = Counter() # Key = ngram order, and value = no. of ngram matches.
    p_denominators = Counter() # Key = ngram order, and value = no. of ngram in ref.
    hyp_lengths, ref_lengths = 0, 0

    assert len(list_of_references) == len(hypotheses), \
            "The number of hypotheses and their reference(s) should be the same"

    # Iterate through each hypothesis and their corresponding references.
    for references, hypothesis in zip(list_of_references, hypotheses):
        assert type(hypothesis.ngram) == dict and \
                all(type(reference.ngram) == dict for reference in references)
        # For each order of ngram, calculate the numerator and
        # denominator for the corpus-level modified precision.
        for i, _ in enumerate(weights, start=1):
            p_i = modified_precision_amr(references, hypothesis, i)
            if p_i == None:
                continue
            p_numerators[i] += p_i.numerator
            p_denominators[i] += p_i.denominator

        # Calculate the hypothesis length and the closest reference length.
        # Adds them to the corpus-level hypothesis and reference counts.
        hyp_len =  hypothesis.length
        hyp_lengths += hyp_len
        ref_lengths += closest_ref_length_amr(references, hyp_len)

    # Calculate corpus-level brevity penalty.
    bp = brevity_penalty(ref_lengths, hyp_lengths)

    # Uniformly re-weighting based on maximum hypothesis lengths if largest
    # order of n-grams < 4 and weights is set at default.
    if auto_reweigh:
        max_gram = max([x for x,y in p_denominators.items() if y > 0])
        if max_gram < len(weights):
            weights = ( 1.0 / max_gram ,) * max_gram
            print('Auto_reweigh, max-gram is', max_gram, 'new weight is', weights)

    # Collects the various precision values for the different ngram orders.
    p_n = [Fraction(p_numerators[i], p_denominators[i], _normalize=False)
           for i, _ in enumerate(weights, start=1)]

    # Returns 0 if there's no matching n-grams
    # We only need to check for p_numerators[1] == 0, since if there's
    # no unigrams, there won't be any higher order ngrams.
    if p_numerators[1] == 0:
        return 0

    # If there's no smoothing, set use method0 from SmoothinFunction class.
    if not smoothing_function:
        smoothing_function = SmoothingFunction().method0
    # Smoothen the modified precision.
    # Note: smoothing_function() may convert values into floats;
    #       it tries to retain the Fraction object as much as the
    #       smoothing method allows.
    p_n = smoothing_function(p_n, references=references, hypothesis=hypothesis,
                             hyp_len=hyp_len, emulate_multibleu=emulate_multibleu)
    s = (w * math.log(p_i) for i, (w, p_i) in enumerate(list(zip(weights, p_n))))
    s =  bp * math.exp(math.fsum(s))
    return round(s, 4) if emulate_multibleu else s

def sentence_bleu(references, hypothesis, weights=(0.34, 0.33, 0.33),
                  smoothing_function=None, auto_reweigh=False,
                  emulate_multibleu=False):
    return corpus_bleu([references], [hypothesis],
                        weights, smoothing_function, auto_reweigh,
                        emulate_multibleu)


sntbleu = round(sentence_bleu([thisRef], thisHyp, weights=weights, smoothing_function=smoofunc, auto_reweigh=False), 1)
print(sntbleu)

0.6


In [ ]:
# physics examples

dialogs = [
        [
            {"role": "system", "content": "Decompose the teacher's instructions into a list of numbered main events. Also decompose each event into its main arguments (ARG) and event modifiers (MOD) that help answer 'Who did what to whom when where and why?' Also provide lists of event relations, main events, and associated social attitudes."},
            {"role": "user", "content": "Teacher: So today, you're going to be setting up another pulley in the lab, um, to gather a little more data to, um, solidify your understanding of pulleys. So far you've set up 4 - or you've done some, um, testing with 4 different pulleys set up, okay. Does anyone remember what those names are without looking of the 4 different pulleys set up?"},
            {"role": "assistant", "content": """Main events:\n1. So today, you're going to be setting up another pulley in the lab,
Event=(e1, set-up)
ARG=(z1, "you", actor)
ARG=(z2, "another pulley", undergoer)
MOD=(z3, "today", time)
MOD=(z4, "lab", location)

2. to gather a little more data
Event=(e2, gather)
ARG=(z5, "you", actor)
ARG=(z6, "more data", theme)

3. your understanding of pulleys
Event=(e3, understand)
ARG=(z5, "you", experiencer)
ARG=(z7, "pulleys", stimulus)

4. to, um, solidify your understanding of pulleys
Event=(e4, solidify)
ARG=(z5, "you", actor)
ARG=(z8, e3, theme)

5. So far you've set up 4,
Event=(e5, set-up)
ARG=(z5, "you", actor)
ARG=(z9, "4 pulleys", undergoer)

6. 4 different pulleys set up
Event=(e6, set-up)
ARG=(z10, "4 different pulleys set up", undergoer)

7. you've done some, um, testing with 4 different pulleys set up
Event=(e7, test)
ARG=(z5, "you", agent)
ARG=(z10, "4 different pulleys set up", undergoer)

8. names of the 4 different pulleys set up
Event=(e8, name)
ARG=(z10, "4 different pulleys set up", named)

9. without looking 
Event=(e9, look)
ARG=(z5, "you", experiencer)
ARG=(z10, e8, stimulus)
POLARITY=(z11, "-")

10. Does anyone remember what those names are of the 4 different pulleys set up?
Event=(e10, remember)
ARG=(z12, unknown, experiencer),
ARG=(z10, e8, stimulus)
MOD=(z13, e9, manner)

Relationships:
Relation=(e1, have-purpose, e2)
Relation=(e1, have-purpose, e4)
Relation=(e3, precondition, e1)
Relation=(e4, scalar-increase, e3)
Relation=(e5, time-before, e1)
Relation=(e6, precondition, e7)
Relation=(e7, time-before, e1)
Relation=(e8, property, z9)
Relation=(e10, time-overlap, e9)
Relation=(z12, subset-of, z5)

Central events: e1, e10
e1: plan for today
e10: request for information

Social attitudes:
Mutual belief
Shared plans
Obligations"""},
            {"role": "user", "content": "Teacher: Why. Right? It's not what you chose out of 4 multiple choice options, it's why did you choose that. Why does that option make the most sense."}
        ],
        [
            {"role": "system", "content": "Decompose the teacher's instructions into a list of numbered main events. Also decompose each event into its main arguments (ARG) and event modifiers (MOD) that help answer 'Who did what to whom when where and why?' Also provide lists of event relations, main events, and associated social attitudes."},
            {"role": "user", "content": f"Teacher: So today, you're going to be setting up another pulley in the lab, um, to gather a little more data to, um, solidify your understanding of pulleys. So far you've set up 4 - or you've done some, um, testing with 4 different pulleys set up, okay. Does anyone remember what those names are without looking of the 4 different pulleys set up?\nMeaning representation:{instances[1][2]}"},
            {"role": "assistant", "content": """Main events:\n1. So today, you're going to be setting up another pulley in the lab,
                    Event=(e1, set-up)
                    ARG=(z1, "you", actor)
                    ARG=(z2, "another pulley", undergoer)
                    MOD=(z3, "today", time)
                    MOD=(z4, "lab", location)

                    2. to gather a little more data
                    Event=(e2, gather)
                    ARG=(z5, "you", actor)
                    ARG=(z6, "more data", theme)

                    3. your understanding of pulleys
                    Event=(e3, understand)
                    ARG=(z5, "you", experiencer)
                    ARG=(z7, "pulleys", stimulus)

                    4. to, um, solidify your understanding of pulleys
                    Event=(e4, solidify)
                    ARG=(z5, "you", actor)
                    ARG=(z8, e3, theme)

                    5. So far you've set up 4,
                    Event=(e5, set-up)
                    ARG=(z5, "you", actor)
                    ARG=(z9, "4 pulleys", undergoer)

                    6. 4 different pulleys set up
                    Event=(e6, set-up)
                    ARG=(z10, "4 different pulleys set up", undergoer)

                    7. you've done some, um, testing with 4 different pulleys set up
                    Event=(e7, test)
                    ARG=(z5, "you", agent)
                    ARG=(z10, "4 different pulleys set up", undergoer)

                    8. names of the 4 different pulleys set up
                    Event=(e8, name)
                    ARG=(z10, "4 different pulleys set up", named)

                    9. without looking 
                    Event=(e9, look)
                    ARG=(z5, "you", experiencer)
                    ARG=(z10, e8, stimulus)
                    POLARITY=(z11, "-")

                    10. Does anyone remember what those names are of the 4 different pulleys set up?
                    Event=(e10, remember)
                    ARG=(z12, unknown, experiencer),
                    ARG=(z10, e8, stimulus)
                    MOD=(z13, e9, manner)

                    Relationships:
                    Relation=(e1, have-purpose, e2)
                    Relation=(e1, have-purpose, e4)
                    Relation=(e3, precondition, e1)
                    Relation=(e4, scalar-increase, e3)
                    Relation=(e5, time-before, e1)
                    Relation=(e6, precondition, e7)
                    Relation=(e7, time-before, e1)
                    Relation=(e8, property, z9)
                    Relation=(e10, time-overlap, e9)
                    Relation=(z12, subset-of, z5)

                    Central events: e1, e10
                    e1: plan for today
                    e10: request for information

                    Social attitudes:
                    Mutual belief
                    Shared plans
                    Obligations"""},
            {"role": "user", "content": f"Teacher: Why. Right? It's not what you chose out of 4 multiple choice options, it's why did you choose that. Why does that option make the most sense.\nMeaning representation: {instances[0][2]}"}
        ]
]

In [ ]:
dialogs[0]

In [ ]:
write_path = Path('../data/llama-pulley-prompts.json')


if write_path:
    with open(write_path, 'w') as fout:
        json.dump(dialogs, fout, indent=4)